This case is a simple static cantilever beam, used as a test case by Geradin and Cardona in their book Flexible Multibody Dynamics.

The beam length is $L = 5$ m and its mass per unit length is $\bar{m} = 100$ kg/m. Its cross-sectional inertia is $J = 10$ kgm. Stiffness is: $EA = 4.8e8$ N, $GA = 3.231e8$ N, $GJ = 1e6$ N, $EI = 9.346e6$ Nm.

In [3]:
# Necessary packages and imports
import h5py as h5
import numpy as np
import os
import configparser

In [4]:
# parameters
length = 5
m_bar = 100
j = 10

ea = 4.8e8
ga = 3.231e8
gj = 1e6
ei = 9.346e6